In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 1. 엑셀 데이터 불러오기
df1 = pd.read_excel("./data/1_600.xlsx")
df2 = pd.read_excel("./data/601_1187.xlsx")
df = pd.concat([df1, df2], ignore_index=True)

# 2. 데이터 정제 (3행부터)
df_cleaned = df.iloc[2:].reset_index(drop=True)
columns = ["num1", "num2", "num3", "num4", "num5", "num6", "bonus"]
numbers = df_cleaned[["Unnamed: 13", "Unnamed: 14", "Unnamed: 15",
                      "Unnamed: 16", "Unnamed: 17", "Unnamed: 18", "Unnamed: 19"]].copy()
numbers.columns = columns
numbers = numbers.apply(pd.to_numeric, errors='coerce').dropna().astype(int)

# 3. 입력 데이터 구성
sequence_length = 5
X, y = [], []
for i in range(len(numbers) - sequence_length):
    X.append(numbers.iloc[i:i+sequence_length][columns[:-1]].values)
    y.append(numbers.iloc[i+sequence_length][columns[:-1]].values)
X = np.array(X)
y = np.array(y)

# 4. 정규화
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X.reshape(-1, 6)).reshape(-1, sequence_length, 6)
y_scaled = scaler.transform(y)

# 5. LSTM 모델 정의
model = Sequential([
    LSTM(64, input_shape=(sequence_length, 6)),
    Dense(64, activation='relu'),
    Dense(6)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_scaled, y_scaled, epochs=50, batch_size=16, verbose=1)

# 6. 마지막 5회차로 다음 번호 예측
latest_input = numbers.iloc[-sequence_length:][columns[:-1]].values
latest_scaled = scaler.transform(latest_input).reshape(1, sequence_length, 6)
predicted = scaler.inverse_transform(model.predict(latest_scaled)).flatten()
predicted_numbers = np.clip(np.round(predicted).astype(int), 1, 45)
print("🎯 예측된 다음 회차 번호:", sorted(predicted_numbers))


Epoch 1/50


C:\Users\qkral\anaconda3\envs\env1\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


74/74 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0759
Epoch 2/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0390
Epoch 3/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0382
Epoch 4/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0383
Epoch 5/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0382
Epoch 6/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0377
Epoch 7/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0371
Epoch 8/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0371
Epoch 9/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0371
Epoch 10/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0371
Epoch 11/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0370
Epoch 12/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0367
Epoch 13/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0367
Epoch 14/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0369
Epoch 15/50
74/74 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0372
Epoch 16/50
74/74 ━━━━━━━━━━━━